# We want to access data in LIMS, but how?

In [121]:
import pg8000          #pg8000 access SQL databases
import pandas as pd    #pandas will be needed to work in a dataframe
import csv
import os
import pathlib2
import warnings
import numpy as np
warnings.filterwarnings('ignore')

### Helpful functions for accessing LIMS

In [32]:
#code from Agata
#these are nice functions to open LIMS, make a query and then close LIMS after

def _connect(user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    conn = pg8000.connect(user=user, host=host, database=database, password=password, port=port)
    return conn, conn.cursor()

def _select(cursor, query):
    cursor.execute(query)
    columns = [ d[0] for d in cursor.description ]
    return [ dict(zip(columns, c)) for c in cursor.fetchall() ]

def limsquery(query, user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    """A function that takes a string containing a SQL query, connects to the LIMS database and outputs the result."""
    conn, cursor = _connect(user, host, database, password, port)
    try:
        results = _select(cursor, query)
    finally:
        
        #THESE ARE IMPORTANT!!!!!!
        #Every query needs to be closed when done
        cursor.close()             
        conn.close()
    return results


#this last function will take our query results and put them in a dataframe so that they are easy to work with
def get_lims_dataframe(query):
    '''Return a dataframe with lims query'''
    result = limsquery(query)
    try:
        data_df = pd.DataFrame(data=result, columns=result[0].keys())
    except IndexError:
        print "Could not find results for your query."
        data_df = pd.DataFrame()
    return data_df

### What are the elements named in LIMS?

#### This is an easy way to search through the tables in LIMS to find out what columns are called

In [75]:

my_query = "SELECT * FROM donors LIMIT 1"
#we are going to select all columns in the specimens table and limit our search to the first 10

my_result = limsquery(my_query)

first_element = my_result
#now we only want to look at the first element of our result

print sorted(first_element)
#print my_result

['age_id', 'baseline_weight_g', 'created_at', 'created_by', 'data', 'date_of_birth', 'death_cause_id', 'death_manner_id', 'death_on', 'education_level_id', 'external_donor_name', 'full_genotype', 'gender_id', 'handedness_id', 'height', 'id', 'induction_method', 'name', 'occupation_id', 'organism_id', 'primary_tissue_source_id', 'race_id', 'transgenic_induction_method_id', 'updated_at', 'updated_by', 'weight']


In [80]:
my_query = "SELECT * FROM specimens LIMIT 1"
#we are going to select all columns in the specimens table and limit our search to the first 10

my_result = limsquery(my_query)

first_element = my_result[0]
#now we only want to look at the first element of our result

print sorted(first_element)

['alignment3d_id', 'barcode', 'biophysical_model_state', 'carousel_well_name', 'cell_depth', 'cell_label', 'cell_prep_id', 'cell_reporter_id', 'cortex_layer_id', 'created_at', 'created_by', 'data', 'donor_id', 'ephys_cell_plan_id', 'ephys_neural_tissue_plan_id', 'ephys_qc_result', 'ephys_roi_result_id', 'ephys_start_time_sec', 'external_specimen_name', 'facs_well_id', 'flipped_specimen_id', 'frozen_at', 'hemisphere_id', 'histology_well_name', 'id', 'location_id', 'name', 'normalization_group_id', 'operation_id', 'parent_id', 'parent_x_coord', 'parent_y_coord', 'parent_z_coord', 'patched_cell_container', 'pinned_radius', 'plane_of_section_id', 'postmortem_interval_id', 'preparation_method_id', 'priority', 'project_id', 'reference_space_id', 'rna_integrity_number', 'specimen_preparation_method_id', 'specimen_set_id', 'storage_directory', 'structure_id', 'task_flow_id', 'tissue_ph', 'tissue_processing_id', 'updated_at', 'updated_by', 'x_coord', 'y_coord']


### There is a lot to process there, but it looks like a dictionary. Let's search another table, but this time only look at the keys so we know what info LIMS is giving us

In [179]:
my_query = "SELECT * FROM ephys_roi_results LIMIT 1"
my_result = limsquery(my_query)
#my_result = get_lims_dataframe(my_query)
first_element = my_result[0]
print first_element.keys()
#print my_result
#lims_df = get_lims_dataframe(my_query)
#lims_df.head()

['rig_name', 'ephys_qc_criteria_id', 'failed_bad_rs', 'updated_at', 'storage_directory', 'electrode_0_pa', 'input_resistance_mohm', 'id', 'stage2_reviewer_id', 'blowout_mv', 'failed_other', 'sampling_rate', 'input_access_resistance_ratio', 'failed_no_seal', 'workflow_state', 'ephys_specimen_roi_plan_id', 'initial_access_resistance_mohm', 'qc_notes', 'recording_date', 'created_at', 'seal_gohm', 'published_at', 'failed_clogged_pipette', 'stage1_reviewer_id', 'failed_electrode_0', 'notes']


## What other tables are out there?

In [36]:
my_query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES"
my_result = limsquery(my_query)
my_result

[{'table_name': u'pg_type'},
 {'table_name': u'pg_roles'},
 {'table_name': u'pg_group'},
 {'table_name': u'pg_user'},
 {'table_name': u'pg_rules'},
 {'table_name': u'pg_views'},
 {'table_name': u'pg_tables'},
 {'table_name': u'pg_matviews'},
 {'table_name': u'pg_indexes'},
 {'table_name': u'pg_stats'},
 {'table_name': u'pg_settings'},
 {'table_name': u'pg_locks'},
 {'table_name': u'pg_cursors'},
 {'table_name': u'pg_available_extensions'},
 {'table_name': u'pg_available_extension_versions'},
 {'table_name': u'pg_prepared_xacts'},
 {'table_name': u'pg_prepared_statements'},
 {'table_name': u'pg_seclabels'},
 {'table_name': u'pg_timezone_abbrevs'},
 {'table_name': u'pg_timezone_names'},
 {'table_name': u'pg_stat_all_tables'},
 {'table_name': u'pg_stat_xact_all_tables'},
 {'table_name': u'pg_stat_sys_tables'},
 {'table_name': u'pg_stat_xact_sys_tables'},
 {'table_name': u'pg_stat_user_tables'},
 {'table_name': u'pg_stat_xact_user_tables'},
 {'table_name': u'pg_statio_all_tables'},
 {'tabl

### We can start to find data now and linking between tables


This is the basics of how a SQL query looks

SELECT TableA., TableB., TableC., TableD. FROM TableA JOIN TableB ON TableB.aID = TableA.aID JOIN TableC ON TableC.cID = TableB.cID WHERE DATE(TableC.date)=date(now())

This allows us to pull in info from 4 different tables since everything we want is not stored in one place, but there are overlaps to link things up. 10 digit IDs generally are your link

In [37]:
lims_query = "SELECT ephys_roi_results.id, specimens.id, specimens.name \
FROM ephys_roi_results JOIN specimens ON specimens.ephys_roi_result_id = ephys_roi_results.id"
lims_df = get_lims_dataframe(lims_query)
lims_df.tail()

,id,name
14142,673724358,Ndnf-IRES2-dgCre;Ai14-379866.03.01.01
14143,673753671,Slc32a1-IRES-Cre;Ai14-379843.03.02.01
14144,673761791,Ndnf-IRES2-dgCre;Ai14-379866.03.01.02
14145,673764347,Ndnf-IRES2-dgCre;Ai14-379866.04.02.01
14146,673769249,Slc32a1-IRES-Cre;Ai14-379843.03.02.02


#### This is the structure we want to follow for SQL:

#### SELECT your tables.columns

#### FROM these columns are coming if you are going to starting using

#### JOIN left ON right

#### WHERE gives you the ability to filter data

#### AND and OR add variability to your filters

#### " opens and closes your query"

#### \ lets you wrap text

### We can shorten ephys_roi_results to err and specimens to s, search for all of our features and use 

### JOIN to join the search results from ephys_roi_results and specimens following the outline from above

In [40]:
# callout the abreveations in the FROM section
# ephys_roi_results err

lims_query = "SELECT err.id, s.id AS cell_id, s.name \
FROM ephys_roi_results err \
JOIN specimens s ON s.ephys_roi_result_id = err.id"

lims_df = get_lims_dataframe(lims_query)
lims_df.tail()

,cell_id,id,name
14142,673724358,673724343,Ndnf-IRES2-dgCre;Ai14-379866.03.01.01
14143,673753671,673753609,Slc32a1-IRES-Cre;Ai14-379843.03.02.01
14144,673761791,673761757,Ndnf-IRES2-dgCre;Ai14-379866.03.01.02
14145,673764347,673764332,Ndnf-IRES2-dgCre;Ai14-379866.04.02.01
14146,673769249,673769228,Slc32a1-IRES-Cre;Ai14-379843.03.02.02


## Let's add another table and filter data

In [39]:
#callout the abreveations in the FROM section
# ephys_roi_results err

lims_query = "SELECT err.id, s.id, s.name, proj.code \
FROM ephys_roi_results err JOIN specimens s ON s.ephys_roi_result_id = err.id \
JOIN projects proj ON s.project_id = proj.id \
WHERE proj.code = 'H301'"

lims_df = get_lims_dataframe(lims_query)
lims_df.tail()

,code,id,name
908,H301,643584484,H17.06.013.12.04.01
909,H301,643589588,H17.06.013.12.04.02
910,H301,643601251,H17.06.013.12.13.01
911,H301,643619982,H17.06.013.12.13.02
912,H301,643625553,H17.06.013.12.13.03


## Things can only get more complicated from here

In [136]:
lims_query = "SELECT err.id, err.recording_date, err.failed_bad_rs, \
err.failed_electrode_0, err.failed_clogged_pipette, err.failed_no_seal, \
err.failed_other, err.workflow_state, s.name, s.ephys_roi_result_id, s.project_id, proj.id, proj.code \
FROM ephys_roi_results err JOIN specimens s ON s.ephys_roi_result_id = err.id \
JOIN projects proj ON s.project_id = proj.id \
WHERE (failed_bad_rs = 'TRUE' \
AND err.failed_electrode_0 = 'FALSE' \
AND err.failed_clogged_pipette = 'FALSE' \
AND err.failed_no_seal = 'FALSE' \
AND err.failed_other = 'FALSE' \
AND err.workflow_state != 'manual_passed') \
AND (proj.code = 'T301' OR proj.code = 'T301x')"

lims_df = get_lims_dataframe(lims_query)
lims_df

,ephys_roi_result_id,code,recording_date,failed_bad_rs,workflow_state,name,failed_clogged_pipette,failed_other,project_id,failed_electrode_0,id,failed_no_seal
0,321023255,T301,2015-01-16 15:47:12,True,manual_failed,Pvalb-IRES-Cre;Ai14-171555.05.02.01,False,False,305094322,False,305094322,False
1,322498292,T301,2015-01-22 11:18:24,True,manual_failed,Sst-IRES-Cre;Ai14-172535.03.01.01,False,False,305094322,False,305094322,False
2,547274305,T301,2016-09-26 15:58:16,True,manual_failed,Nos1-CreERT2;Sst-IRES-FlpO;Ai65-271577.06.01.02,False,False,305094322,False,305094322,False
3,560411917,T301,2016-12-01 13:18:22,True,manual_failed,Vip-IRES-Cre;Ai14(IVSCC)-284133.04.01.01,False,False,305094322,False,305094322,False
4,562182500,T301,2016-12-22 14:02:19,True,manual_failed,Htr3a-Cre_NO152;Ai14-288787.04.02.01,False,False,305094322,False,305094322,False
5,562724516,T301,2017-01-06 14:20:36,True,manual_failed,Vip-IRES-Cre;Ai14-290642.03.02.01,False,False,305094322,False,305094322,False
6,563367964,T301,2017-01-11 14:09:16,True,manual_failed,Nkx2-1-CreERT2;Ai14 (IVSCC)-291491.04.02.01,False,False,305094322,False,305094322,False
7,563594967,T301,2017-01-12 14:02:43,True,manual_failed,Nkx2-1-CreERT2;Ai14 (IVSCC)-291490.03.01.01,False,False,305094322,False,305094322,False
8,563596846,T301,2017-01-12 14:18:01,True,manual_failed,Nkx2-1-CreERT2;Ai14 (IVSCC)-291490.04.01.01,False,False,305094322,False,305094322,False
9,564789700,T301,2017-01-17 14:00:53,True,manual_failed,Ndnf-IRES2-dgCre;Ai14-292674.04.02.01,False,False,305094322,False,305094322,False


# Other Useful Queries

#### look for all the multipatch cells

In [42]:
lims_query = "SELECT err.id, err.recording_date, s.name, s.ephys_roi_result_id, proj.code \
FROM ephys_roi_results err JOIN specimens s ON s.ephys_roi_result_id = err.id \
JOIN projects proj ON s.project_id = proj.id \
AND proj.code = 'mMPATCH'"

lims_df = get_lims_dataframe(lims_query)
lims_df.tail()

,ephys_roi_result_id,code,id,recording_date,name
255,673174542,mMPATCH,673174542,None,Vip-IRES-Cre;Ai140;Sst-IRES-FlpO;Ai65F-379033....
256,673178228,mMPATCH,673178228,None,Nr5a1-Cre;Ai14-380236.09.06.01
257,673179586,mMPATCH,673179586,None,Nr5a1-Cre;Ai14-380236.09.06.02
258,673179600,mMPATCH,673179600,None,Nr5a1-Cre;Ai14-380236.10.06.01
259,673181959,mMPATCH,673181959,None,Nr5a1-Cre;Ai14-380236.08.06.01


#### find the Feature Json files all cells that failed QC only due to high access resistance

In [43]:
lims_query = "SELECT err.id, err.recording_date, err.failed_bad_rs, \
err.failed_electrode_0, err.failed_clogged_pipette, err.failed_no_seal, \
err.failed_other, err.workflow_state, s.name, s.ephys_roi_result_id, s.project_id, proj.id, proj.code, \
f.storage_directory, f.attachable_id \
FROM ephys_roi_results err JOIN specimens s ON s.ephys_roi_result_id = err.id \
JOIN projects proj ON s.project_id = proj.id \
JOIN well_known_files f ON err.id = f.attachable_id \
WHERE (failed_bad_rs = 'TRUE' \
AND err.failed_electrode_0 = 'FALSE' \
AND err.failed_clogged_pipette = 'FALSE' \
AND err.failed_no_seal = 'FALSE' \
AND err.failed_other = 'FALSE' \
AND err.workflow_state != 'manual_passed') \
AND (proj.code = 'T301' OR proj.code = 'T301x') \
AND f.filename LIKE '%%ephys_features.json'"

lims_df = get_lims_dataframe(lims_query)
lims_df.tail()

,ephys_roi_result_id,code,recording_date,failed_bad_rs,workflow_state,name,attachable_id,failed_clogged_pipette,storage_directory,failed_other,project_id,failed_electrode_0,id,failed_no_seal
40,579497253,T301,2017-04-04 15:22:26,True,manual_failed,Vipr2-IRES2-Cre;Ai14-310516.06.02.01,579497253,False,/allen/programs/celltypes/production/mousecell...,False,305094322,False,305094322,False
41,579645792,T301,2017-04-05 12:56:54,True,manual_failed,Ndnf-IRES2-dgCre;Ai14-311164.05.02.01,579645792,False,/allen/programs/celltypes/production/mousecell...,False,305094322,False,305094322,False
42,580812829,T301,2017-04-11 09:44:52,True,manual_failed,Nkx2-1-CreERT2;Ai14 (IVSCC)-311670.04.02.01,580812829,False,/allen/programs/celltypes/production/mousecell...,False,305094322,False,305094322,False
43,580900050,T301,2017-04-11 14:44:26,True,manual_failed,Chrna2-Cre_OE25;Pvalb-T2A-Dre;Ai66-309757.05.0...,580900050,False,/allen/programs/celltypes/production/mousecell...,False,305094322,False,305094322,False
44,580901813,T301,2017-04-11 15:21:05,True,manual_failed,Chrna2-Cre_OE25;Pvalb-T2A-Dre;Ai66-309757.05.0...,580901813,False,/allen/programs/celltypes/production/mousecell...,False,305094322,False,305094322,False


#### find patchseq cells entered in LIMS after Jan. 1, 2017 and their RNA results

In [44]:
lims_query = "SELECT ra.amplified_quantity_ng, ra.name, ra.run_date, ra.percent_cdna_longer_than_400bp, ra.failed, \
ra.cycles, ra.id, rai.sample_id, cell.id, cell.name, cell.created_at, cell.patched_cell_container \
FROM specimens cell \
LEFT JOIN rna_amplification_inputs rai ON rai.sample_id = cell.id \
LEFT JOIN rna_amplifications ra ON ra.id = rai.rna_amplification_id \
WHERE run_date > '2017-01-01'"

lims_df = get_lims_dataframe(lims_query)
lims_df.tail()

,amplified_quantity_ng,name,run_date,percent_cdna_longer_than_400bp,created_at,failed,patched_cell_container,sample_id,cycles,id
3615,10.035892,Ndnf-IRES2-dgCre;Slc32a1-IRES2-FlpO;Ai65-37810...,2018-09-03 07:00:00,0.357,2018-03-07 19:15:41.807285,True,P2S4_180307_051_A01,670589879,20,670589879
3616,11.284884,Ndnf-IRES2-dgCre;Slc32a1-IRES2-FlpO;Ai65-37810...,2018-09-03 07:00:00,0.688,2018-03-07 21:00:37.056067,False,P2S4_180307_052_A01,670666224,20,670666224
3617,1.961381,Crh-IRES-Cre_ZJH;Sst-IRES-FlpO;Ai65-378262.03....,2018-09-03 07:00:00,0.337,2018-03-08 00:35:38.513578,True,P2S4_180307_053_A01,670702807,20,670702807
3618,4.036444,Ndnf-IRES2-dgCre;Slc32a1-IRES2-FlpO;Ai65-37810...,2018-09-03 07:00:00,0.271,2018-03-07 18:30:36.136546,True,P8S4_180307_351_A01,670539587,20,670539587
3619,1.539159,Ndnf-IRES2-dgCre;Slc32a1-IRES2-FlpO;Ai65-37810...,2018-09-03 07:00:00,0.317,2018-03-07 18:55:33.062179,True,P8S4_180307_352_A01,670568833,20,670568833


#### find the transgenic line for each cell

In [45]:
lims_query = "SELECT specimens.name AS cell_name, donors.id, specimens.donor_id, specimens.external_specimen_name, donors.full_genotype, donors.name, specimens.patched_cell_container \
FROM donors JOIN specimens ON specimens.donor_id = donors.id \
WHERE specimens.ephys_roi_result_id IS NOT NULL"

lims_df = get_lims_dataframe(lims_query)
lims_df

,name,full_genotype,patched_cell_container,donor_id,id,external_specimen_name,cell_name
0,H16.03.002,None,None,520276747,520276747,None,H16.03.002.01.06.02
1,Tlx3-Cre_PL56;Ai140;Vip-IRES-FlpO;Ai65F-378317,Tlx3-Cre_PL56/wt;Ai140(TIT2L-GFP-ICL-tTA2)/wt,None,667489991,667489991,None,Tlx3-Cre_PL56;Ai140;Vip-IRES-FlpO;Ai65F-378317...
2,Slc32a1-IRES2-FlpO;Ai65F-363946,Slc32a1-IRES2-FlpO/wt;Ai65F/wt,P8S4_171219_352_A01,652361174,652361174,None,Slc32a1-IRES2-FlpO;Ai65F-363946.04.02.02
3,Rorb-IRES2-Cre;Ai14-355893,Rorb-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,P8S4_171023_356_A01,644234836,644234836,None,Rorb-IRES2-Cre;Ai14-355893.04.02.01
4,Rorb-IRES2-Cre;Ai14-355894,Rorb-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,P8S4_171025_351_A01,644234847,644234847,None,Rorb-IRES2-Cre;Ai14-355894.04.02.01
5,Rorb-IRES2-Cre;Ai14-355894,Rorb-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,P8S4_171025_353_A01,644234847,644234847,None,Rorb-IRES2-Cre;Ai14-355894.04.02.03
6,Tlx3-Cre_PL56;Ai140;Vip-IRES-FlpO;Ai65F-378317,Tlx3-Cre_PL56/wt;Ai140(TIT2L-GFP-ICL-tTA2)/wt,None,667489991,667489991,None,Tlx3-Cre_PL56;Ai140;Vip-IRES-FlpO;Ai65F-378317...
7,Cux2-CreERT2;Ai14-362883,Cux2-CreERT2/wt;Ai14(RCL-tdT)/wt,P9S4_171208_403_A01,650868847,650868847,None,Cux2-CreERT2;Ai14-362883.03.01.01
8,Chrna2-Cre_OE25;Ai14-362810,Chrna2-Cre_OE25/wt;Ai14(RCL-tdT)/wt,PAS4_171207_454_A01,650869055,650869055,None,Chrna2-Cre_OE25;Ai14-362810.05.01.02
9,Chrna2-Cre_OE25;Ai14-362810,Chrna2-Cre_OE25/wt;Ai14(RCL-tdT)/wt,PAS4_171207_455_A01,650869055,650869055,None,Chrna2-Cre_OE25;Ai14-362810.05.01.03


## How can I easily add to another dataframe (like the metadata in the Google Doc)?

#### df_new = pd.merge(left = df_1, right = df_2, left_on = column_from_left, right_on = column_from_right, how = 'inner')

In [14]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://shanelynnwebsite-mid9n9g1q9y8tt.netdna-ssl.com/wp-content/uploads/2017/03/join-types-merge-names.jpg")

In [46]:
home = pathlib2.Path.home()
full_path = home/"documents"/"stash"/"ps-metadata-analysis"/"data"/"patch_seq_log_mouse.csv"
full_path = str(full_path)
full_path
initial = pd.read_csv(full_path)

In [47]:
ps = initial[['cell type', 'File', 'Post patch?', 'Post patch pipette R' , 'PCR cycles', 'SM_QC_PF', 'Bad dates','Lims tube id']]
#ps["Slice (Lims ID?)"].str.strip()


In [48]:
ps['Bad dates'] = ps['Bad dates'].astype('category')
ps['PCR cycles'] = ps['PCR cycles'].astype('category')
ps['cell type'] = ps['cell type'].astype('category')
ps['Post patch?'] = ps['Post patch?'].astype('category')
ps = ps[ps['Bad dates'] != 'x']
ps = ps[ps['SM_QC_PF'] != 'cnt']
ps = ps[ps['cell type'] == 'tdt+']
ps.dropna(subset = ['Post patch?', 'SM_QC_PF'], inplace = True)

ps = ps.drop('Bad dates', 1)
ps = ps.drop('SM_QC_PF', 1)
ps = ps.drop('PCR cycles', 1)
ps = ps.drop('cell type', 1)
ps.head()

,File,Post patch?,Post patch pipette R,Lims tube id
70,Rbp4-Cre_KL100;Ai14-290392.04.02.01,Outside-Out,NaN,P2S4_170109_051_A01
71,Rbp4-Cre_KL100;Ai14-290392.04.02.02,Outside-Out,NaN,P2S4_170109_052_A01
77,Rbp4-Cre_KL100;Ai14-290393.03.02.01,Outside-Out,NaN,P2S4_170110_051_A01
78,Rbp4-Cre_KL100;Ai14-290393.03.02.02,Outside-Out,NaN,P2S4_170110_052_A01
79,Rbp4-Cre_KL100;Ai14-290393.04.02.01,No-Seal,NaN,P2S4_170110_053_A01


In [49]:
new_df = pd.merge(left = ps, right = lims_df, left_on = 'File', right_on = 'cell_name', how = 'left')

In [50]:
new_df

,File,Post patch?,Post patch pipette R,Lims tube id,name,full_genotype,patched_cell_container,donor_id,id,external_specimen_name,cell_name
0,Rbp4-Cre_KL100;Ai14-290392.04.02.01,Outside-Out,NaN,P2S4_170109_051_A01,Rbp4-Cre_KL100;Ai14-290392,Rbp4-Cre_KL100/wt;Ai14(RCL-tdT)/wt,P2S4_170109_051_A01,562588746.0,562588746.0,None,Rbp4-Cre_KL100;Ai14-290392.04.02.01
1,Rbp4-Cre_KL100;Ai14-290392.04.02.02,Outside-Out,NaN,P2S4_170109_052_A01,Rbp4-Cre_KL100;Ai14-290392,Rbp4-Cre_KL100/wt;Ai14(RCL-tdT)/wt,P2S4_170109_052_A01,562588746.0,562588746.0,None,Rbp4-Cre_KL100;Ai14-290392.04.02.02
2,Rbp4-Cre_KL100;Ai14-290393.03.02.01,Outside-Out,NaN,P2S4_170110_051_A01,Rbp4-Cre_KL100;Ai14-290393,Rbp4-Cre_KL100/wt;Ai14(RCL-tdT)/wt,P2S4_170110_051_A01,562588779.0,562588779.0,None,Rbp4-Cre_KL100;Ai14-290393.03.02.01
3,Rbp4-Cre_KL100;Ai14-290393.03.02.02,Outside-Out,NaN,P2S4_170110_052_A01,Rbp4-Cre_KL100;Ai14-290393,Rbp4-Cre_KL100/wt;Ai14(RCL-tdT)/wt,P2S4_170110_052_A01,562588779.0,562588779.0,None,Rbp4-Cre_KL100;Ai14-290393.03.02.02
4,Rbp4-Cre_KL100;Ai14-290393.04.02.01,No-Seal,NaN,P2S4_170110_053_A01,Rbp4-Cre_KL100;Ai14-290393,Rbp4-Cre_KL100/wt;Ai14(RCL-tdT)/wt,P2S4_170110_053_A01,562588779.0,562588779.0,None,Rbp4-Cre_KL100;Ai14-290393.04.02.01
5,Rbp4-Cre_KL100;Ai14-290393.04.02.02,Outside-Out,NaN,P2S4_170110_054_A01,Rbp4-Cre_KL100;Ai14-290393,Rbp4-Cre_KL100/wt;Ai14(RCL-tdT)/wt,P2S4_170110_054_A01,562588779.0,562588779.0,None,Rbp4-Cre_KL100;Ai14-290393.04.02.02
6,Rbp4-Cre_KL100;Ai14-290393.04.02.03,Outside-Out,NaN,P2S4_170110_055_A01,Rbp4-Cre_KL100;Ai14-290393,Rbp4-Cre_KL100/wt;Ai14(RCL-tdT)/wt,P2S4_170110_055_A01,562588779.0,562588779.0,None,Rbp4-Cre_KL100;Ai14-290393.04.02.03
7,Rbp4-Cre_KL100;Ai14-290393.04.02.04,Outside-Out,NaN,P2S4_170110_056_A01,Rbp4-Cre_KL100;Ai14-290393,Rbp4-Cre_KL100/wt;Ai14(RCL-tdT)/wt,P2S4_170110_056_A01,562588779.0,562588779.0,None,Rbp4-Cre_KL100;Ai14-290393.04.02.04
8,Gad2-IRES-Cre;Ai14-293418.06.01.01,No-Seal,NaN,P1S4_170111_001_A01,Gad2-IRES-Cre;Ai14-293418,Gad2-IRES-Cre/wt;Ai14(RCL-tdT)/wt,P1S4_170111_001_A01,562377197.0,562377197.0,None,Gad2-IRES-Cre;Ai14-293418.06.01.01
9,Gad2-IRES-Cre;Ai14-293418.06.01.02,Partial-Nucleus,NaN,P1S4_170111_002_A01,Gad2-IRES-Cre;Ai14-293418,Gad2-IRES-Cre/wt;Ai14(RCL-tdT)/wt,P1S4_170111_002_A01,562377197.0,562377197.0,None,Gad2-IRES-Cre;Ai14-293418.06.01.02


In [51]:
lims_query = "SELECT name, barcode FROM specimens \
WHERE specimens.ephys_roi_result_id IS NOT NULL"

lims_df = get_lims_dataframe(lims_query)
lims_df.tail()

,barcode,name
14142,0643511562,Chrna2-Cre_OE25;Ai14-355016.05.02.01
14143,0643518188,Chrna2-Cre_OE25;Ai14-355016.05.02.02
14144,0670343682,Pvalb-IRES-Cre;Ai14-378390.04.02.01
14145,0669864819,Nr5a1-Cre;Ai14-378059.10.06.01
14146,0669865467,Nr5a1-Cre;Ai14-378059.11.06.01


In [52]:
lims_query = "SELECT specimens.name as cell_name, specimens.barcode, specimens.donor_id, donors.id \
FROM specimens JOIN donors ON specimens.donor_id = donors.id \
WHERE specimens.ephys_roi_result_id IS NOT NULL"

df = get_lims_dataframe(lims_query)
df.tail()

,barcode,donor_id,id,cell_name
14142,0643511562,642979415,642979415,Chrna2-Cre_OE25;Ai14-355016.05.02.01
14143,0643518188,642979415,642979415,Chrna2-Cre_OE25;Ai14-355016.05.02.02
14144,0670343682,668456824,668456824,Pvalb-IRES-Cre;Ai14-378390.04.02.01
14145,0669864819,667489947,667489947,Nr5a1-Cre;Ai14-378059.10.06.01
14146,0669865467,667489947,667489947,Nr5a1-Cre;Ai14-378059.11.06.01


In [53]:
lims_query = "SELECT cell.name as cell_name, donors.full_genotype, cell.donor_id, donors.id \
FROM specimens cell JOIN donors ON cell.donor_id = donors.id \
WHERE cell.ephys_roi_result_id IS NOT NULL"

lims_df = get_lims_dataframe(lims_query)
lims_df.tail()

,donor_id,id,full_genotype,cell_name
14142,642979415,642979415,Chrna2-Cre_OE25/wt;Ai14(RCL-tdT)/wt,Chrna2-Cre_OE25;Ai14-355016.05.02.01
14143,642979415,642979415,Chrna2-Cre_OE25/wt;Ai14(RCL-tdT)/wt,Chrna2-Cre_OE25;Ai14-355016.05.02.02
14144,668456824,668456824,Pvalb-IRES-Cre/wt;Ai14(RCL-tdT)/wt,Pvalb-IRES-Cre;Ai14-378390.04.02.01
14145,667489947,667489947,Nr5a1-Cre/wt;Ai14(RCL-tdT)/wt,Nr5a1-Cre;Ai14-378059.10.06.01
14146,667489947,667489947,Nr5a1-Cre/wt;Ai14(RCL-tdT)/wt,Nr5a1-Cre;Ai14-378059.11.06.01


### How many cells took longer to find instantaneous threshold than they did to find rheobase(ie more SSFINEST than LSFINEST sweeps

In [66]:
lims_query = "SELECT specimens.name AS cell_name, \
COUNT (CASE WHEN ephys_stimuli.description LIKE '%%C1SSFINEST150112%%' THEN 1 ELSE NULL END) AS shortsquare, \
COUNT (CASE WHEN ephys_stimuli.description LIKE '%%C1LSFINEST150112%%' THEN 1 ELSE NULL END) AS longsquare \
FROM specimens JOIN ephys_sweeps ON specimens.id = ephys_sweeps.specimen_id \
JOIN ephys_stimuli ON ephys_sweeps.ephys_stimulus_id = ephys_stimuli.id \
WHERE specimens.patched_cell_container NOTNULL \
GROUP BY specimens.name"

df = get_lims_dataframe(lims_query)
df = df.query('shortsquare > longsquare') 
len(df)
#df.head()

3393

### Show the distribution of hemispheres all the cells patched in 2017 with the age and sex of the mice used.

In [131]:
lims_query = "SELECT specimens.name AS cell_name, err.recording_date AS date, donors.age_id, donors.gender_id \
FROM specimens JOIN donors ON specimens.donor_id = donors.id \
JOIN ephys_roi_results err ON specimens.ephys_roi_result_id = err.id \
WHERE specimens.ephys_roi_result_id IS NOT NULL and err.recording_date IS NOT NULL"

df = get_lims_dataframe(lims_query)
df['Year'] = pd.DatetimeIndex(df['date']).year
df = df[df['Year'] == 2017]
df.loc[df.cell_name.str.endswith('.01'), 'Hemi'] = 'Left'
df.loc[df.cell_name.str.endswith('.02'), 'Hemi'] = 'Right'
df.tail()
#df.dtypes

,date,gender_id,age_id,cell_name,Year,Hemi
12260,2017-12-21 21:00:30,1,298414239,Slc32a1-IRES-Cre;Ai14-364558.05.01.02,2017,Right
12261,2017-12-21 22:16:49,1,298414239,Chat-IRES-Cre-neo;Ai14-364380.02.02.01,2017,Left
12262,2017-12-21 22:22:33,1,298414239,Chat-IRES-Cre-neo;Ai14-364380.05.02.01,2017,Left
12263,2017-12-22 19:50:35,2,111,Chat-IRES-Cre-neo;Ai14-364381.04.02.01,2017,Left
12264,2017-12-22 20:20:19,2,111,Chat-IRES-Cre-neo;Ai14-364381.04.02.02,2017,Right


### Look for cells that do not have a qc reviewer and failed qc for project T301. What are the cell level qc failures? (access, gohm seal, blowout, input resistance)

In [188]:
lims_query = "SELECT err.workflow_state, err.stage1_reviewer_id, err.stage2_reviewer_id, \
err.blowout_mv, err.failed_bad_rs, err.failed_no_seal, err. failed_electrode_0, err.failed_clogged_pipette, \
err.failed_other, projects.code \
FROM ephys_roi_results err JOIN specimens ON specimens.ephys_roi_result_id = err.id \
JOIN projects ON specimens.project_id = projects.id \
WHERE (stage1_reviewer_id IS NULL \
AND err.stage2_reviewer_id IS NULL) \
AND (err.workflow_state = 'manual_failed' OR err.workflow_state = 'auto_failed') \
AND projects.code = 'T301'"



lims_df = get_lims_dataframe(lims_query)
lims_df.head()

,code,failed_bad_rs,workflow_state,stage2_reviewer_id,blowout_mv,stage1_reviewer_id,failed_other,failed_electrode_0,failed_clogged_pipette,failed_no_seal
0,T301,False,auto_failed,None,NaN,None,None,True,False,None
1,T301,False,auto_failed,None,NaN,None,None,False,False,None
2,T301,False,auto_failed,None,NaN,None,None,True,False,None
3,T301,False,auto_failed,None,NaN,None,None,True,False,None
4,T301,False,auto_failed,None,NaN,None,None,False,False,None


### Find all the H301 cells where DiJon was stage 1 reviewer that manually passed qc and the cells where Lisa was stage 2 review that manually failed qc.

In [204]:
lims_query = "SELECT err.workflow_state, err.stage1_reviewer_id, err.stage2_reviewer_id, projects.code \
FROM ephys_roi_results err JOIN specimens ON specimens.ephys_roi_result_id = err.id \
JOIN projects ON specimens.project_id = projects.id \
WHERE projects.code = 'H301' \
AND (err.stage1_reviewer_id = '305127608' AND err.workflow_state = 'manual_passed') \
AND (err.stage2_reviewer_id = '485591509' AND err.workflow_state = 'manual_failed')"

lims_df = get_lims_dataframe(lims_query)
lims_df

Could not find results for your query.


""
